# Imports

In [ ]:
%matplotlib inline

In [ ]:
import tensorflow as tf
import tensorflow_docs as tfdocs
import tensorflow_docs.modeling
import sys
from pathlib import Path
import datetime
import time
import numpy as np
import pandas as pd
from pprint import pprint
from typing import Dict, Any, Union, List, Tuple
from functools import partial
import re
import string
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.metrics import confusion_matrix
from math import ceil
from collections import namedtuple
from sklearn.model_selection import train_test_split
import pickle

import chakin
import json
import os
from collections import defaultdict
import zipfile
import sqlite3
import logging
from tempfile import TemporaryDirectory
from fastprogress import progress_bar
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import NearMiss
from imblearn.over_sampling import RandomOverSampler

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Embedding
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import concatenate
from tensorflow.keras.regularizers import l2
from tensorflow.keras.regularizers import L2
import tensorflow as tf
import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from keras.regularizers import l2

from sklearn.model_selection import KFold
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import SGDClassifier

from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import NearMiss
from imblearn.over_sampling import RandomOverSampler

import matplotlib.pyplot as plt
import seaborn as sns
import random
#from EmbeddingFactory import EmbeddingFactory
from sklearn.metrics import roc_auc_score, classification_report, precision_score, recall_score, auc,average_precision_score, f1_score, accuracy_score, precision_recall_curve

from numpy.random import seed
pd.set_option('display.max_rows', None)

print(f"Using Tensorflow, {tf.__version__} on Python interpreter, {sys.version_info}")

# RANDOM SET SEED

In [ ]:
RANDOM_SEED = 47568
seed(47568)
#tf.random.set_random_seed(seed_value)
os.environ['PYTHONHASHSEED']=str(47568)
random.seed(47568)
tf.random.set_seed(47568)

SAVE = False

np.random.seed(RANDOM_SEED)

print(f"Using random seed, {RANDOM_SEED}")

# Load Data

In [ ]:
DATA_PATH = '/Users/sikha/0FAIRFL/Data/113p/'

In [ ]:
X_train = np.load(DATA_PATH+"X_train.npy",allow_pickle=True)
y_train = np.load(DATA_PATH+"y_train.npy",allow_pickle=True)
X_test = np.load(DATA_PATH+"X_test.npy",allow_pickle=True)
y_test = np.load(DATA_PATH+"y_test.npy",allow_pickle=True)

In [ ]:
test_df = pd.read_pickle(DATA_PATH+"testdf.pkl")
train_df = pd.read_pickle(DATA_PATH+"traindf.pkl")

# Sensitive attribute

In [ ]:
# Single Binary sensitive attribute and Binary classification

In [ ]:
SENS_ATTR = 'Female' 
# gender column OHE

LABEL_NAME = 'Rating'
PROT_ATTR_NAME = 'Female'
UNPROT_ATTR_NAME = 'Male'


# MODELS

## Central -SKLEARN

In [ ]:
lr_model = LogisticRegression(random_state = RANDOM_SEED, solver='sag',max_iter=10000)
lr_model.fit(X_train,y_train.ravel())
y_prob = lr_model.predict_proba(X_test)


In [ ]:
print(lr_model.n_iter_)
print("ROC AUC: ", roc_auc_score(y_test.ravel(), y_prob[:,1]))
precision, recall, _ = precision_recall_curve(y_test.ravel(), y_prob[:,1])
auc_score = auc(recall, precision)
print('PR AUC: %.3f' % auc_score)
print("AP: ", average_precision_score(y_test.ravel(), y_prob[:,1]))
tn, fp, fn, tp = confusion_matrix(y_test,np.argmax(y_prob, axis = 1).ravel(), labels=[0, 1]).ravel()
print("TPR OA:", tp/(tp+fn))

In [ ]:
test_df['pred'] = np.argmax(y_prob, axis = 1)
test_df['pred'] = test_df['pred'].astype('int')

prot_df = test_df[test_df[PROT_ATTR_NAME] == 1][[LABEL_NAME,'pred']]
tn, fp, fn, tp = confusion_matrix(prot_df[[LABEL_NAME]].values.ravel(),prot_df[['pred']].values.ravel(), labels=[0, 1]).ravel()
prot_tpr = tp/(tp+fn)
print(PROT_ATTR_NAME,":", tp/(tp+fn))

unprot_df = test_df[test_df[UNPROT_ATTR_NAME] == 1][[LABEL_NAME,'pred']]
tn, fp, fn, tp = confusion_matrix(unprot_df[[LABEL_NAME]].values.ravel(),unprot_df[['pred']].values.ravel(), labels=[0, 1]).ravel()
unprot_tpr = tp/(tp+fn)
print(UNPROT_ATTR_NAME,":", tp/(tp+fn))

print("DI:", max(prot_tpr/unprot_tpr, unprot_tpr/prot_tpr))


## Central - KERAS

In [ ]:
# T have similarity and better perf 2 units so reshaping y_train

In [ ]:
y_train_2 = []
for label in y_train:
    if label == 1:
        x = [0,1]
    else:
        x = [1,0]
    y_train_2.append(np.array(x))
y_train_2 = np.array(y_train_2)

In [ ]:
keras_model = Sequential()
keras_model.add(Dense(units=2,kernel_initializer='glorot_uniform', activation='sigmoid',kernel_regularizer=l2(0.)))
keras_model.compile(optimizer='sgd',
                  loss='binary_crossentropy',
                  metrics=[tf.keras.metrics.Recall()])
hist = keras_model.fit(X_train, y_train_2,batch_size=64, epochs=630)#validation_split=0.2)


In [ ]:
y_prob_keras  = keras_model.predict(X_test)


In [ ]:
print("ROC AUC: ", roc_auc_score(y_test.ravel(), y_prob_keras[:,1]))

In [ ]:
sum(np.argmax(y_prob_m2, axis = 1))

In [ ]:
keras_model.save(DATA_PATH+"model")

In [ ]:
test_df['pred_k'] = np.argmax(y_prob_keras, axis = 1)
test_df['pred_k'] = test_df['pred_k'].astype('int')

In [ ]:

prot_df = test_df[test_df[PROT_ATTR_NAME] == 1][[LABEL_NAME,'pred_k']]
tn, fp, fn, tp = confusion_matrix(prot_df[[LABEL_NAME]].values.ravel(),prot_df[['pred_k']].values.ravel(), labels=[0, 1]).ravel()
prot_tpr = tp/(tp+fn)
print(PROT_ATTR_NAME,":", tp/(tp+fn))

unprot_df = test_df[test_df[UNPROT_ATTR_NAME] == 1][[LABEL_NAME,'pred_k']]
tn, fp, fn, tp = confusion_matrix(unprot_df[[LABEL_NAME]].values.ravel(),unprot_df[['pred_k']].values.ravel(), labels=[0, 1]).ravel()
unprot_tpr = tp/(tp+fn)
print(UNPROT_ATTR_NAME,":", tp/(tp+fn))

print("DI:", max(prot_tpr/unprot_tpr, unprot_tpr/prot_tpr))

## Reweighing techniques - collect stats

In [ ]:

#count(s=female)
cs1 = train_df[PROT_ATTR_NAME].sum()
#count(s=female)
cs0 = train_df[UNPROT_ATTR_NAME].sum()
#count(y=1)
cy1 = train_df[LABEL_NAME].sum()
#count(y=0)
cy0 = train_df.shape[0] - cy1

print(cs1, cs0)
print(cy1, cy0)

cs0y0 = train_df[(train_df[PROT_ATTR_NAME]==0) & (train_df[LABEL_NAME]==0)].shape[0]
cs0y1 = train_df[(train_df[PROT_ATTR_NAME]==0) & (train_df[LABEL_NAME]==1)].shape[0]
cs1y0 = train_df[(train_df[PROT_ATTR_NAME]==1) & (train_df[LABEL_NAME]==0)].shape[0]
cs1y1 = train_df[(train_df[PROT_ATTR_NAME]==1) & (train_df[LABEL_NAME]==1)].shape[0]
print(cs0y0,cs0y1)
print(cs1y0,cs1y1)
tot = X_train.shape[0]
    
    



## FairBalanceClass
### 1 / count(y and a)

In [ ]:

def assignweights_fb(x):
    if x[PROT_ATTR_NAME] == 0 and x[LABEL_NAME] == 0:
        return 1/cs0y0
    if x[PROT_ATTR_NAME] == 0 and x[LABEL_NAME] == 1:
        return 1/cs0y1
    if x[PROT_ATTR_NAME] == 1 and x[LABEL_NAME] == 0:
        return 1/cs1y0
    if x[PROT_ATTR_NAME] == 1 and x[LABEL_NAME] == 1:
        return 1/cs1y1  

train_df['weights_fb'] = train_df[[PROT_ATTR_NAME,LABEL_NAME]].apply(assignweights_fb, axis=1)
sample_weights = train_df['weights_fb'].values.ravel() 
sample_weights = sample_weights * len(y_train.ravel()) / sum(sample_weights)    


fb_model = LogisticRegression(random_state = RANDOM_SEED, solver='sag',max_iter=10000)
fb_model.fit(X_train,y_train.ravel(),sample_weights)
y_prob_fb_bal = fb_model.predict_proba(X_test)
print(lr_model_fb_bal.n_iter_)
print("ROC AUC: %.3f" %  roc_auc_score(y_test.ravel(), y_prob_fb_bal[:,1]))

test_df['pred_fb_bal'] = np.argmax(y_prob_fb_bal, axis = 1)
test_df['pred_fb_bal'] = test_df['pred_fb_bal'].astype('int')

prot_df = test_df[test_df[PROT_ATTR_NAME] == 1][[LABEL_NAME,'pred_fb_bal']]
tn, fp, fn, tp = confusion_matrix(prot_df[[LABEL_NAME]].values.ravel(),prot_df[['pred_fb_bal']].values.ravel(), labels=[0, 1]).ravel()
prot_tpr = tp/(tp+fn)
print(PROT_ATTR_NAME,":", tp/(tp+fn))

unprot_df = test_df[test_df[UNPROT_ATTR_NAME] == 1][[LABEL_NAME,'pred_fb_bal']]
tn, fp, fn, tp = confusion_matrix(unprot_df[[LABEL_NAME]].values.ravel(),unprot_df[['pred_fb_bal']].values.ravel(), labels=[0, 1]).ravel()
unprot_tpr = tp/(tp+fn)
print(UNPROT_ATTR_NAME,":", tp/(tp+fn))

print("DI:", max(prot_tpr/unprot_tpr, unprot_tpr/prot_tpr))


In [ ]:
model_b = Sequential()
model_b.add(Dense(units=2,kernel_initializer='glorot_uniform', activation='sigmoid',kernel_regularizer=l2(0.)))
model_b.compile(optimizer='sgd',
                  loss='binary_crossentropy',
                  metrics=[tf.keras.metrics.Recall()])
model_b.fit(X_train, y_train_2,sample_weight=sample_weights,batch_size=64, epochs=700)#validation_split=0.2)

In [ ]:
y_prob_fb_bal_k  = model_b2.predict(X_test)
print("ROC AUC: ", roc_auc_score(y_test.ravel(), y_prob_fb_bal_k[:,1]))


In [ ]:

test_df['pred_fb_bal'] = np.argmax(y_prob_fb_bal_k, axis = 1)
test_df['pred_fb_bal'] = test_df['pred_fb_bal'].astype('int')

prot_df = test_df[test_df[PROT_ATTR_NAME] == 1][[LABEL_NAME,'pred_fb_bal']]
tn, fp, fn, tp = confusion_matrix(prot_df[[LABEL_NAME]].values.ravel(),prot_df[['pred_fb_bal']].values.ravel(), labels=[0, 1]).ravel()
prot_tpr = tp/(tp+fn)
print(PROT_ATTR_NAME,":", tp/(tp+fn))

unprot_df = test_df[test_df[UNPROT_ATTR_NAME] == 1][[LABEL_NAME,'pred_fb_bal']]
tn, fp, fn, tp = confusion_matrix(unprot_df[[LABEL_NAME]].values.ravel(),unprot_df[['pred_fb_bal']].values.ravel(), labels=[0, 1]).ravel()
unprot_tpr = tp/(tp+fn)
print(UNPROT_ATTR_NAME,":", tp/(tp+fn))

print("DI:", max(prot_tpr/unprot_tpr, unprot_tpr/prot_tpr))

## FairBalance
### count(y) / count(y and a)

In [ ]:


def assignweights_fb(x):
    if x[PROT_ATTR_NAME] == 0 and x[LABEL_NAME] == 0:
        return cy0/cs0y0
    if x[PROT_ATTR_NAME] == 0 and x[LABEL_NAME] == 1:
        return cy1/cs0y1
    if x[PROT_ATTR_NAME] == 1 and x[LABEL_NAME] == 0:
        return cy0/cs1y0
    if x[PROT_ATTR_NAME] == 1 and x[LABEL_NAME] == 1:
        return cy1/cs1y1  

    
train_df['weights_fb'] = train_df[[PROT_ATTR_NAME,LABEL_NAME]].apply(assignweights_fb, axis=1)
sample_weights = train_df['weights_fb'].values.ravel() 
sample_weights = sample_weights * len(y_train.ravel()) / sum(sample_weights)    


fb_model = LogisticRegression(random_state = RANDOM_SEED, solver='sag',max_iter=10000)
fb_model.fit(X_train,y_train.ravel(),sample_weights)
y_prob_fb_bal = fb_model.predict_proba(X_test)
print(lr_model_fb_bal.n_iter_)
print("ROC AUC: %.3f" %  roc_auc_score(y_test.ravel(), y_prob_fb_bal[:,1]))

test_df['pred_fb_bal'] = np.argmax(y_prob_fb_bal, axis = 1)
test_df['pred_fb_bal'] = test_df['pred_fb_bal'].astype('int')

prot_df = test_df[test_df[PROT_ATTR_NAME] == 1][[LABEL_NAME,'pred_fb_bal']]
tn, fp, fn, tp = confusion_matrix(prot_df[[LABEL_NAME]].values.ravel(),prot_df[['pred_fb_bal']].values.ravel(), labels=[0, 1]).ravel()
prot_tpr = tp/(tp+fn)
print(PROT_ATTR_NAME,":", tp/(tp+fn))

unprot_df = test_df[test_df[UNPROT_ATTR_NAME] == 1][[LABEL_NAME,'pred_fb_bal']]
tn, fp, fn, tp = confusion_matrix(unprot_df[[LABEL_NAME]].values.ravel(),unprot_df[['pred_fb_bal']].values.ravel(), labels=[0, 1]).ravel()
unprot_tpr = tp/(tp+fn)
print(UNPROT_ATTR_NAME,":", tp/(tp+fn))

print("DI:", max(prot_tpr/unprot_tpr, unprot_tpr/prot_tpr))

## FairBalance2
### 1 / count(a)

In [ ]:
def assignweights_fb(x):
    if x[PROT_ATTR_NAME] == 0 and x[LABEL_NAME] == 0:
        return 1/(cs0)
    if x[PROT_ATTR_NAME] == 0 and x[LABEL_NAME] == 1:
        return 1/(cs0)
    if x[PROT_ATTR_NAME] == 1 and x[LABEL_NAME] == 0:
        return 1/(cs1)
    if x[PROT_ATTR_NAME] == 1 and x[LABEL_NAME] == 1:
        return 1/(cs1)  

    
train_df['weights_fb'] = train_df[[PROT_ATTR_NAME,LABEL_NAME]].apply(assignweights_fb, axis=1)
sample_weights = train_df['weights_fb'].values.ravel() 
sample_weights = sample_weights * len(y_train.ravel()) / sum(sample_weights)    


fb_model = LogisticRegression(random_state = RANDOM_SEED, solver='sag',max_iter=10000)
fb_model.fit(X_train,y_train.ravel(),sample_weights)
y_prob_fb_bal = fb_model.predict_proba(X_test)
print(lr_model_fb_bal.n_iter_)
print("ROC AUC: %.3f" %  roc_auc_score(y_test.ravel(), y_prob_fb_bal[:,1]))

test_df['pred_fb_bal'] = np.argmax(y_prob_fb_bal, axis = 1)
test_df['pred_fb_bal'] = test_df['pred_fb_bal'].astype('int')

prot_df = test_df[test_df[PROT_ATTR_NAME] == 1][[LABEL_NAME,'pred_fb_bal']]
tn, fp, fn, tp = confusion_matrix(prot_df[[LABEL_NAME]].values.ravel(),prot_df[['pred_fb_bal']].values.ravel(), labels=[0, 1]).ravel()
prot_tpr = tp/(tp+fn)
print(PROT_ATTR_NAME,":", tp/(tp+fn))

unprot_df = test_df[test_df[UNPROT_ATTR_NAME] == 1][[LABEL_NAME,'pred_fb_bal']]
tn, fp, fn, tp = confusion_matrix(unprot_df[[LABEL_NAME]].values.ravel(),unprot_df[['pred_fb_bal']].values.ravel(), labels=[0, 1]).ravel()
unprot_tpr = tp/(tp+fn)
print(UNPROT_ATTR_NAME,":", tp/(tp+fn))

print("DI:", max(prot_tpr/unprot_tpr, unprot_tpr/prot_tpr))

## FairBalanceClass2
### 1 / (count(y) * count(a))

In [ ]:
def assignweights_fb(x):
    if x[PROT_ATTR_NAME] == 0 and x[LABEL_NAME] == 0:
        return 1/(cs0 * cy0)
    if x[PROT_ATTR_NAME] == 0 and x[LABEL_NAME] == 1:
        return 1/(cs0 * cy1)
    if x[PROT_ATTR_NAME] == 1 and x[LABEL_NAME] == 0:
        return 1/(cs1 * cy0)
    if x[PROT_ATTR_NAME] == 1 and x[LABEL_NAME] == 1:
        return 1/(cs1 * cy1) 

    
train_df['weights_fb'] = train_df[[PROT_ATTR_NAME,LABEL_NAME]].apply(assignweights_fb, axis=1)
sample_weights = train_df['weights_fb'].values.ravel() 
sample_weights = sample_weights * len(y_train.ravel()) / sum(sample_weights)    


fb_model = LogisticRegression(random_state = RANDOM_SEED, solver='sag',max_iter=10000)
fb_model.fit(X_train,y_train.ravel(),sample_weights)
y_prob_fb_bal = fb_model.predict_proba(X_test)
print(lr_model_fb_bal.n_iter_)
print("ROC AUC: %.3f" %  roc_auc_score(y_test.ravel(), y_prob_fb_bal[:,1]))

test_df['pred_fb_bal'] = np.argmax(y_prob_fb_bal, axis = 1)
test_df['pred_fb_bal'] = test_df['pred_fb_bal'].astype('int')

prot_df = test_df[test_df[PROT_ATTR_NAME] == 1][[LABEL_NAME,'pred_fb_bal']]
tn, fp, fn, tp = confusion_matrix(prot_df[[LABEL_NAME]].values.ravel(),prot_df[['pred_fb_bal']].values.ravel(), labels=[0, 1]).ravel()
prot_tpr = tp/(tp+fn)
print(PROT_ATTR_NAME,":", tp/(tp+fn))

unprot_df = test_df[test_df[UNPROT_ATTR_NAME] == 1][[LABEL_NAME,'pred_fb_bal']]
tn, fp, fn, tp = confusion_matrix(unprot_df[[LABEL_NAME]].values.ravel(),unprot_df[['pred_fb_bal']].values.ravel(), labels=[0, 1]).ravel()
unprot_tpr = tp/(tp+fn)
print(UNPROT_ATTR_NAME,":", tp/(tp+fn))

print("DI:", max(prot_tpr/unprot_tpr, unprot_tpr/prot_tpr))

## IBM paper
### Reweighing technique from 2012 paper

In [ ]:


ws0y0 = ((cs0/tot) * (cy0/tot)) / (cs0y0/tot)
ws0y1 = ((cs0/tot) * (cy1/tot)) / (cs0y1/tot)
ws1y0 = ((cs1/tot) * (cy0/tot)) / (cs1y0/tot)
ws1y1 = ((cs1/tot) * (cy1/tot)) / (cs1y1/tot)

print(ws0y0,ws0y1,ws1y0,ws1y1)

def assignweights(x):
    if x[PROT_ATTR_NAME] == 0 and x[LABEL_NAME] == 0:
        return ws0y0 #* tot/(2*cy0) 
    if x[PROT_ATTR_NAME] == 0 and x[LABEL_NAME] == 1:
        return ws0y1 #* tot/(2*cy1)
    if x[PROT_ATTR_NAME] == 1 and x[LABEL_NAME] == 0:
        return ws1y0 #* tot/(2*cy0)
    if x[PROT_ATTR_NAME] == 1 and x[LABEL_NAME] == 1:
        return ws1y1 #* tot/(2*cy1)  
    
train_df['weights_fb'] = train_df[[PROT_ATTR_NAME,LABEL_NAME]].apply(assignweights_fb, axis=1)
sample_weights = train_df['weights_fb'].values.ravel() 

fb_model = LogisticRegression(random_state = RANDOM_SEED, solver='sag',max_iter=10000)
fb_model.fit(X_train,y_train.ravel(),sample_weights)
y_prob_fb_bal = fb_model.predict_proba(X_test)
print(lr_model_fb_bal.n_iter_)
print("ROC AUC: %.3f" %  roc_auc_score(y_test.ravel(), y_prob_fb_bal[:,1]))

test_df['pred_fb_bal'] = np.argmax(y_prob_fb_bal, axis = 1)
test_df['pred_fb_bal'] = test_df['pred_fb_bal'].astype('int')

prot_df = test_df[test_df[PROT_ATTR_NAME] == 1][[LABEL_NAME,'pred_fb_bal']]
tn, fp, fn, tp = confusion_matrix(prot_df[[LABEL_NAME]].values.ravel(),prot_df[['pred_fb_bal']].values.ravel(), labels=[0, 1]).ravel()
prot_tpr = tp/(tp+fn)
print(PROT_ATTR_NAME,":", tp/(tp+fn))

unprot_df = test_df[test_df[UNPROT_ATTR_NAME] == 1][[LABEL_NAME,'pred_fb_bal']]
tn, fp, fn, tp = confusion_matrix(unprot_df[[LABEL_NAME]].values.ravel(),unprot_df[['pred_fb_bal']].values.ravel(), labels=[0, 1]).ravel()
unprot_tpr = tp/(tp+fn)
print(UNPROT_ATTR_NAME,":", tp/(tp+fn))

print("DI:", max(prot_tpr/unprot_tpr, unprot_tpr/prot_tpr))

## Sklearn class balance
### tot / 2*count(y)

In [ ]:


fb_model = LogisticRegression(random_state = RANDOM_SEED,class_weight='balanced', solver='sag',max_iter=10000)
fb_model.fit(X_train,y_train.ravel())
y_prob_fb_bal = fb_model.predict_proba(X_test)
print(lr_model_fb_bal.n_iter_)
print("ROC AUC: %.3f" %  roc_auc_score(y_test.ravel(), y_prob_fb_bal[:,1]))

test_df['pred_fb_bal'] = np.argmax(y_prob_fb_bal, axis = 1)
test_df['pred_fb_bal'] = test_df['pred_fb_bal'].astype('int')

prot_df = test_df[test_df[PROT_ATTR_NAME] == 1][[LABEL_NAME,'pred_fb_bal']]
tn, fp, fn, tp = confusion_matrix(prot_df[[LABEL_NAME]].values.ravel(),prot_df[['pred_fb_bal']].values.ravel(), labels=[0, 1]).ravel()
prot_tpr = tp/(tp+fn)
print(PROT_ATTR_NAME,":", tp/(tp+fn))

unprot_df = test_df[test_df[UNPROT_ATTR_NAME] == 1][[LABEL_NAME,'pred_fb_bal']]
tn, fp, fn, tp = confusion_matrix(unprot_df[[LABEL_NAME]].values.ravel(),unprot_df[['pred_fb_bal']].values.ravel(), labels=[0, 1]).ravel()
unprot_tpr = tp/(tp+fn)
print(UNPROT_ATTR_NAME,":", tp/(tp+fn))

print("DI:", max(prot_tpr/unprot_tpr, unprot_tpr/prot_tpr))

## My method 
### 

In [ ]:
def assignweights_bal(x):
    if x[PROT_ATTR_NAME] == 0 and x[LABEL_NAME] == 0:
        return ws0y0 * tot/(2*cy0) 
    if x[PROT_ATTR_NAME] == 0 and x[LABEL_NAME] == 1:
        return ws0y1 * tot/(2*cy1)
    if x[PROT_ATTR_NAME] == 1 and x[LABEL_NAME] == 0:
        return ws1y0 * tot/(2*cy0)
    if x[PROT_ATTR_NAME] == 1 and x[LABEL_NAME] == 1:
        return ws1y1 * tot/(2*cy1)  
    
train_df['weights_fb'] = train_df[[PROT_ATTR_NAME,LABEL_NAME]].apply(assignweights_fb, axis=1)
sample_weights = train_df['weights_fb'].values.ravel() 

fb_model = LogisticRegression(random_state = RANDOM_SEED,class_weight='balanced', solver='sag',max_iter=10000)
fb_model.fit(X_train,y_train.ravel(),sample_weights)
y_prob_fb_bal = fb_model.predict_proba(X_test)
print(lr_model_fb_bal.n_iter_)
print("ROC AUC: %.3f" %  roc_auc_score(y_test.ravel(), y_prob_fb_bal[:,1]))

test_df['pred_fb_bal'] = np.argmax(y_prob_fb_bal, axis = 1)
test_df['pred_fb_bal'] = test_df['pred_fb_bal'].astype('int')

prot_df = test_df[test_df[PROT_ATTR_NAME] == 1][[LABEL_NAME,'pred_fb_bal']]
tn, fp, fn, tp = confusion_matrix(prot_df[[LABEL_NAME]].values.ravel(),prot_df[['pred_fb_bal']].values.ravel(), labels=[0, 1]).ravel()
prot_tpr = tp/(tp+fn)
print(PROT_ATTR_NAME,":", tp/(tp+fn))

unprot_df = test_df[test_df[UNPROT_ATTR_NAME] == 1][[LABEL_NAME,'pred_fb_bal']]
tn, fp, fn, tp = confusion_matrix(unprot_df[[LABEL_NAME]].values.ravel(),unprot_df[['pred_fb_bal']].values.ravel(), labels=[0, 1]).ravel()
unprot_tpr = tp/(tp+fn)
print(UNPROT_ATTR_NAME,":", tp/(tp+fn))

print("DI:", max(prot_tpr/unprot_tpr, unprot_tpr/prot_tpr))

# SMOTE

In [ ]:
sm = SMOTE(random_state=42)
X_res, y_res = sm.fit_resample(X_train, y_train.ravel())

fb_model = LogisticRegression(random_state = RANDOM_SEED,class_weight='balanced', solver='sag',max_iter=10000)
fb_model.fit(X_res,y_res)
y_prob_fb_bal = fb_model.predict_proba(X_test)
print(lr_model_fb_bal.n_iter_)
print("ROC AUC: %.3f" %  roc_auc_score(y_test.ravel(), y_prob_fb_bal[:,1]))

test_df['pred_fb_bal'] = np.argmax(y_prob_fb_bal, axis = 1)
test_df['pred_fb_bal'] = test_df['pred_fb_bal'].astype('int')

prot_df = test_df[test_df[PROT_ATTR_NAME] == 1][[LABEL_NAME,'pred_fb_bal']]
tn, fp, fn, tp = confusion_matrix(prot_df[[LABEL_NAME]].values.ravel(),prot_df[['pred_fb_bal']].values.ravel(), labels=[0, 1]).ravel()
prot_tpr = tp/(tp+fn)
print(PROT_ATTR_NAME,":", tp/(tp+fn))

unprot_df = test_df[test_df[UNPROT_ATTR_NAME] == 1][[LABEL_NAME,'pred_fb_bal']]
tn, fp, fn, tp = confusion_matrix(unprot_df[[LABEL_NAME]].values.ravel(),unprot_df[['pred_fb_bal']].values.ravel(), labels=[0, 1]).ravel()
unprot_tpr = tp/(tp+fn)
print(UNPROT_ATTR_NAME,":", tp/(tp+fn))

print("DI:", max(prot_tpr/unprot_tpr, unprot_tpr/prot_tpr))

# DO NOT TRY - BELOW

# FairSMOTE

In [ ]:
maximum = max(cs0y0,cs0y1,cs1y0,cs1y1)
print(cs0y0,cs0y1,cs1y0,cs1y1)
print(maximum)
cs0y0_to_inc = maximum - cs0y0
cs0y1_to_inc = maximum - cs0y1
cs1y1_to_inc = maximum - cs1y1

# OTHERS

In [ ]:
def assignweights_plus(x):
    if x.Female == 0 and x.Rating == 0:
        return ws0y0 + tot/(2*cy0) 
    if x.Female == 0 and x.Rating == 1:
        return ws0y1 + tot/(2*cy1)
    if x.Female == 1 and x.Rating == 0:
        return ws1y0 + tot/(2*cy0)
    if x.Female == 1 and x.Rating == 1:
        return ws1y1 + tot/(2*cy1) 

In [ ]:
#count(s=female)
cs1 = train_df['Female'].sum()
#count(s=female)
cs0 = train_df['Male'].sum()
#count(s=old)
ca1 = train_df[train_df['Age_bucket'] == 'old'].shape[0]
#count(s=female)
ca0 = train_df[train_df['Age_bucket'] != 'old'].shape[0]
#count(y=1)
cy1 = train_df['Rating'].sum()
#count(y=0)
cy0 = train_df.shape[0] - cy1

print(cs1, cs0)
print(cy1, cy0)

cs0y0 = train_df[(train_df['Female']==0) & (train_df['Rating']==0)].shape[0]
cs0y1 = train_df[(train_df['Female']==0) & (train_df['Rating']==1)].shape[0]
cs1y0 = train_df[(train_df['Female']==1) & (train_df['Rating']==0)].shape[0]
cs1y1 = train_df[(train_df['Female']==1) & (train_df['Rating']==1)].shape[0]

ca0y0 = train_df[(train_df['Age_bucket'] != 'old') & (train_df['Rating']==0)].shape[0]
ca0y1 = train_df[(train_df['Age_bucket'] != 'old') & (train_df['Rating']==1)].shape[0]
ca1y0 = train_df[(train_df['Age_bucket'] == 'old') & (train_df['Rating']==0)].shape[0]
ca1y1 = train_df[(train_df['Age_bucket'] == 'old') & (train_df['Rating']==1)].shape[0]
print(cs0y0,cs0y1)
print(cs1y0,cs1y1)
tot = X_train.shape[0]

ws0y0 = ((cs0/tot) * (cy0/tot)) / (cs0y0/tot)
ws0y1 = ((cs0/tot) * (cy1/tot)) / (cs0y1/tot)
ws1y0 = ((cs1/tot) * (cy0/tot)) / (cs1y0/tot)
ws1y1 = ((cs1/tot) * (cy1/tot)) / (cs1y1/tot)

wa0y0 = ((ca0/tot) * (cy0/tot)) / (ca0y0/tot)
wa0y1 = ((ca0/tot) * (cy1/tot)) / (ca0y1/tot)
wa1y0 = ((ca1/tot) * (cy0/tot)) / (ca1y0/tot)
wa1y1 = ((ca1/tot) * (cy1/tot)) / (ca1y1/tot)

print(ws0y0,ws0y1,ws1y0,ws1y1)




    
    
#   (rw1+rw2)*bal
def assignweights_multi(x):
    if x.Female == 0 and x.Rating == 0:
        if x.Age_bucket == 'old':
            p = (ws0y0 + wa1y0) * tot/(2*cy0) 
        else:
            p = (ws0y0 + wa0y0) * tot/(2*cy0) 
        return p
    if x.Female == 0 and x.Rating == 1:
        if x.Age_bucket == 'old':
            p = (ws0y1 + wa1y1) * tot/(2*cy1) 
        else:
            p = (ws0y1 + wa0y1) * tot/(2*cy1) 
        return p

    if x.Female == 1 and x.Rating == 0:
        if x.Age_bucket == 'old':
            p = (ws1y0 + wa1y0) * tot/(2*cy0) 
        else:
            p = (ws1y0 + wa0y0) * tot/(2*cy0) 
        return p
    if x.Female == 1 and x.Rating == 1:
        if x.Age_bucket == 'old':
            p = (ws1y1 + wa1y1) * tot/(2*cy1) 
        else:
            p = (ws1y1 + wa0y1) * tot/(2*cy1) 
        return p
    
#   (rw1*rw2)*bal
def assignweights_multi_2(x):
    if x.Female == 0 and x.Rating == 0:
        if x.Age_bucket == 'old':
            p = (ws0y0 * wa1y0) * tot/(2*cy0) 
        else:
            p = (ws0y0 * wa0y0) * tot/(2*cy0) 
        return p
    if x.Female == 0 and x.Rating == 1:
        if x.Age_bucket == 'old':
            p = (ws0y1 * wa1y1) * tot/(2*cy1) 
        else:
            p = (ws0y1 * wa0y1) * tot/(2*cy1) 
        return p

    if x.Female == 1 and x.Rating == 0:
        if x.Age_bucket == 'old':
            p = (ws1y0 * wa1y0) * tot/(2*cy0) 
        else:
            p = (ws1y0 * wa0y0) * tot/(2*cy0) 
        return p
    if x.Female == 1 and x.Rating == 1:
        if x.Age_bucket == 'old':
            p = (ws1y1 * wa1y1) * tot/(2*cy1) 
        else:
            p = (ws1y1 * wa0y1) * tot/(2*cy1) 
        return p
    
    
    
#   (rw1*bal)*(rw2*bal)
def assignweights_multi_2(x):
    if x.Female == 0 and x.Rating == 0:
        if x.Age_bucket == 'old':
            p = (ws0y0 * wa1y0) * tot/(2*cy0) * tot/(2*cy0) 
        else:
            p = (ws0y0 * wa0y0) * tot/(2*cy0) * tot/(2*cy0) 
        return p
    if x.Female == 0 and x.Rating == 1:
        if x.Age_bucket == 'old':
            p = (ws0y1 * wa1y1) * tot/(2*cy1) * tot/(2*cy1)
        else:
            p = (ws0y1 * wa0y1) * tot/(2*cy1) * tot/(2*cy1)
        return p

    if x.Female == 1 and x.Rating == 0:
        if x.Age_bucket == 'old':
            p = (ws1y0 * wa1y0) * tot/(2*cy0) * tot/(2*cy0) 
        else:
            p = (ws1y0 * wa0y0) * tot/(2*cy0) * tot/(2*cy0)
        return p
    if x.Female == 1 and x.Rating == 1:
        if x.Age_bucket == 'old':
            p = (ws1y1 * wa1y1) * tot/(2*cy1) * tot/(2*cy1) 
        else:
            p = (ws1y1 * wa0y1) * tot/(2*cy1) * tot/(2*cy1) 
        return p
    
  #   (rw1)*(rw2)  
def assignweights_multi_3(x):
    if x.Female == 0 and x.Rating == 0:
        if x.Age_bucket == 'old':
            p = (ws0y0 * wa1y0)  
        else:
            p = (ws0y0 * wa0y0) 
        return p
    if x.Female == 0 and x.Rating == 1:
        if x.Age_bucket == 'old':
            p = (ws0y1 * wa1y1) 
        else:
            p = (ws0y1 * wa0y1) 
        return p

    if x.Female == 1 and x.Rating == 0:
        if x.Age_bucket == 'old':
            p = (ws1y0 * wa1y0)  
        else:
            p = (ws1y0 * wa0y0)  
        return p
    if x.Female == 1 and x.Rating == 1:
        if x.Age_bucket == 'old':
            p = (ws1y1 * wa1y1)  
        else:
            p = (ws1y1 * wa0y1) 
        return p
    
